# Age Prediction from Faces

Zooey Nguyen

Predicting age from an image of a face with conditional variational auto-encoder. Using the GTKFaces dataset.

## Log
2022-07-11
- helpful docs
    - "conditional adversarial autoencoder": https://zzutk.github.io/Face-Aging-CAAE/
    - simpler one for face reconstruction: https://arxiv.org/pdf/2112.02139.pdf
    - celebrityA reconstruction: https://goodboychan.github.io/python/coursera/tensorflow_probability/icl/2021/09/14/03-Variational-AutoEncoder-Celeb-A.html
    - vanilla one: https://github.com/podgorskiy/VAE
- using BatchDataset, used map inside it to put images as their own labels
    
2022-07-12

2022-07-13
- https://www.tensorflow.org/tutorials/generative/cvae

## Setup

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

## Data processing

Create a shuffled and batched dataset. 

In [4]:
import os
import h5py
from tensorflow.keras.preprocessing import image_dataset_from_directory

path = "/home/znguyen/UTKFace"
df = image_dataset_from_directory(path, labels=None, batch_size=32)

df = df.map(lambda x: (x, x))
next(iter(df))

Found 23708 files belonging to 1 classes.


(<tf.Tensor: shape=(32, 256, 256, 3), dtype=float32, numpy=
 array([[[[252.       , 254.       , 251.       ],
          [251.32812  , 253.32812  , 250.32812  ],
          [251.45312  , 253.45312  , 250.45312  ],
          ...,
          [230.64062  , 232.64062  , 221.64062  ],
          [231.01562  , 233.34375  , 222.34375  ],
          [229.       , 232.       , 221.       ]],
 
         [[252.       , 254.       , 251.       ],
          [251.32812  , 253.32812  , 250.32812  ],
          [251.14868  , 253.14868  , 250.14868  ],
          ...,
          [230.94507  , 232.33618  , 221.64062  ],
          [231.01562  , 233.34375  , 222.34375  ],
          [229.       , 232.       , 221.       ]],
 
         [[252.       , 254.       , 251.       ],
          [251.32812  , 253.32812  , 250.32812  ],
          [251.       , 253.       , 250.       ],
          ...,
          [230.64062  , 231.73438  , 221.1875   ],
          [230.85986  , 233.0393   , 222.0393   ],
          [229.45312  

## Simple autoencoder

Bare basics.

In [3]:
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D
from tensorflow.keras.layers import InputLayer, Flatten, Dense, Reshape

LATENT_DIM = 16
IMAGE_SHAPE = (256,256,3)

class AutoEncoder(Model):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = Sequential([
            InputLayer(input_shape=IMAGE_SHAPE),
            Conv2D(32, (3,3), activation='relu', padding='same'), 
            Conv2D(32, (3,3), activation='relu', padding='same'),
            Flatten(),
            Dense(LATENT_DIM)
        ])
        self.decoder = Sequential([
            InputLayer(input_shape=(LATENT_DIM,)),
            Dense(units=np.prod(IMAGE_SHAPE), activation='relu'),
            Reshape(target_shape=IMAGE_SHAPE),
            Conv2DTranspose(32, (3,3), activation='relu', padding='same'),
            Conv2DTranspose(32, (3,3), activation='relu', padding='same'),
            Conv2DTranspose(1, (3,3), padding='same')
        ])
        
    def call(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [4]:
model = AutoEncoder()
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(df, epochs=100, verbose=1)

Epoch 1/100


In [ ]:
model.predict(next(iter(df))[0])